# Homework 1 (Total Points: 175)



Learning Goals:
- Learn how to load a dataset and process it.
- Learn how to implement several IR methods (TFIDF, BM25, QL) and understand their weaknesses & strengths.
- Learn how to evaluate IR methods


**NOTE 1**: Only the code (`TODO: Implement this!` denotes these sections) is graded. The 'theory' questions in this assignment serve as a preparation for the exam and to facilitate a deeper understanding of the course content. These questions (denoted by `TODO: Answer this!`) have no points assigned to them, but **need** to be filled out before submission.  

**NOTE 2**: You can use the `nltk`, `numpy` and `matplotlib` libraries here. Other libraries, e.g., `gensim` or `scikit-learn`, may not be used. 

**NOTE 3**: The notebook you submit has to have the student ids, seperated by underscores (E.g., `12341234_12341234_12341234.ipynb`). 

**NOTE 4**: Make sure to check that your notebook runs before submission. A quick way to do this is to restart the kernel and run all the cells.  

---
Additional Resources: 
-  Sections 2.3, 4.1, 4.2, 4.3, 5.3, 5.6, 5.7, 6.2, 7, 8 of [Search Engines: Information Retrieval in Practice](https://ciir.cs.umass.edu/downloads/SEIRiP.pdf)


In [8]:
# imports 
# TODO: Ensure that no additional library is imported in the notebook. 
# TODO: Only the standard library and the following libraries are allowed:

import os
import zipfile
from functools import partial

import nltk
import requests
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt

from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.html import widgets
from collections import namedtuple, defaultdict

%matplotlib inline

## Section 1: Text Processing (20 points)

In this section, we will load the dataset and learn how to clean up the data to make it usable for an IR system. 

We are using the [CACM dataset](http://ir.dcs.gla.ac.uk/resources/test_collections/cacm/), which is a small, classic IR dataset, composed of a collection of titles and abstracts from the journal CACM. It comes with relevance judgements for queries, so we can evaluate our IR system. 

The following cell downloads the dataset and unzips it to a local directory

In [9]:
def download_dataset(folder_path = "./datasets/"):
    
    os.makedirs(folder_path, exist_ok=True)
    
    file_location = os.path.join(folder_path, "cacm.zip")
    
    # download file if it doesn't exist
    if not os.path.exists(file_location):
        
        url = "https://surfdrive.surf.nl/files/index.php/s/M0FGJpX2p8wDwxR/download"

        with open(file_location, "wb") as handle:
            print(f"Downloading file from {url} to {file_location}")
            response = requests.get(url, stream=True)
            for data in tqdm(response.iter_content()):
                handle.write(data)
            print("Finished downloading file")
    
    if not os.path.exists(os.path.join(folder_path, "train.txt")):
        
        # unzip file
        with zipfile.ZipFile(file_location, 'r') as zip_ref:
            zip_ref.extractall(folder_path)
        
download_dataset()

You can see a brief description of each file in the dataset by looking at the README file

In [10]:
##### Read the README file 
!cat ./datasets/README
#####

Files in this directory with sizes:
          0 Jun 19 21:01 README

    2187734 Jun 19 20:55 cacm.all              text of documents
        626 Jun 19 20:58 cite.info             key to citation info
                                                (the X sections in cacm.all)
       2668 Jun 19 20:55 common_words           stop words used by smart
       2194 Jun 19 20:55 make_coll*             shell script to make collection
       1557 Jun 19 20:55 make_coll_term*        ditto (both useless without
                                                smart system)
       9948 Jun 19 20:55 qrels.text             relation giving
                                                    qid did 0 0
                                                to indicate dument did is
                                                relevant to query qid
      13689 Jun 19 20:55 query.text             Original text of the query


----
We are interested in 4 files:
- `cacm.all` : Contains the text for all documents. Note that some documents do not have abstracts available. 
- `query.text` : The text of all queries
- `qrels.text` : The relevance judgements
- `common_words` : A list of common words. This may be used as a collection of stopwords

In [11]:
##### The first 45 lines of the CACM dataset forms the first record
# We are interested only in 3 fields. 
# 1. the '.I' field, which is the document id
# 2. the '.T' field (the title) and
# 3. the '.W' field (the abstract, which may be absent)
!head -45 ./datasets/cacm.all
#####

.I 1
.T
Preliminary Report-International Algebraic Language
.B
CACM December, 1958
.A
Perlis, A. J.
Samelson,K.
.N
CA581203 JB March 22, 1978  8:28 PM
.X
100	5	1
123	5	1
164	5	1
1	5	1
1	5	1
1	5	1
205	5	1
210	5	1
214	5	1
1982	5	1
398	5	1
642	5	1
669	5	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
165	6	1
196	6	1
196	6	1
1273	6	1
1883	6	1
324	6	1
43	6	1
53	6	1
91	6	1
410	6	1
3184	6	1


---

Now, write a function to read in the `cacm.all` file. Note that each document has a variable number of lines. The `.I` field denotes a new document

In [12]:
# TODO: Implement this! (4 points)
def read_cacm_docs(root_folder = "./datasets/"):
    """
        Reads in the CACM documents. The dataset is assumed to be in the folder "./datasets/cacm" be default
        Returns: A list of 2-tuples: (doc_id, document), where 'document' is a single string created by 
            appending the title and abstract (seperated by a "\n"). 
            In case the record doesn't have an abstract, the document is composed only by the title
    """
    
    filepath = os.path.join(root_folder, 'cacm.all')
    doc_ids = []
    documents = []
    
    with open(filepath, 'r') as f:
        docs = f.read().split('.I ')[1:]
    
    for doc in docs:
        lines = doc.split('\n')
        doc_id = int(lines[0])
        title = lines[lines.index('.T')+1]

        if '.W' in lines:
            start = lines.index('.W')+1
            end   = lines.index('.B')
            abstract = '\n'+'  '.join(lines[start:end])

        else:
            abstract = ''
        
        doc_ids.append(doc_id)
        documents.append(title + abstract)
        
    return list(zip(doc_ids, documents))

docs = read_cacm_docs()

In [13]:
##### 
assert len(docs) == 3204, "There should be exactly 3024 documents"
##### 

---

Next, let us read the queries. They are formatted similarly: 

In [14]:
##### The first 15 lines of 'query.text' has 2 queries
# We are interested only in 2 fields. 
# 1. the '.I' - the query id
# 2. the '.W' - the query
!head -15 ./datasets/query.text
#####

.I 1
.W
 What articles exist which deal with TSS (Time Sharing System), an
operating system for IBM computers?
.N
 1. Richard Alexander, Comp Serv, Langmuir Lab (TSS)
 
.I 2
.W
 I am interested in articles written either by Prieve or Udo Pooch
.A
Prieve, B.
Pooch, U.
.N
 2. Richard Alexander, Comp Serv, Langmuir Lab (author = Pooch or Prieve)


---

Now, write a function to read in this file:

In [15]:
# TODO: Implement this! (3 points)
def read_queries(root_folder = "./datasets/"):
    """
        Reads in the CACM queries. The dataset is assumed to be in the folder "./datasets/" be default
        Returns: A list of 2-tuples: (query_id, query)
    """
    
    queries   = []
    filepath = os.path.join(root_folder, 'query.text')
    
    with open(filepath, 'r') as f:
        queries_unprocessed = f.read().split('.I')[1:]
        
    for q in queries_unprocessed:
        lines = q.split('\n')
        
        query_id = int(lines[0])
        
        start = lines.index('.W')+1
        end   = lines.index('.N') if '.A' not in lines else lines.index('.A')
        query = ' '.join(lines[start:end])
        
        queries.append((query_id, query))
        
    return queries
    
    
queries = read_queries()

In [16]:
##### 
assert len(queries) == 64 and all([q[1] is not None for q in queries]), "There should be exactly 64 queries"
##### 

---

Read in the stop words:

In [17]:
!head ./datasets/common_words

a
about
above
accordingly
across
after
afterwards
again
against
all


In [18]:
# TODO: Implement this! (3 points)
def load_stopwords(root_folder = "./datasets"):
    """
    Load the stopwords
    Output: A set of stopwords
    """
    
    filepath = os.path.join(root_folder, 'common_words')
    
    with open(filepath, 'r') as f:
        stopwords = set(f.read().split())
    
    return stopwords

stopwords = load_stopwords()
assert len(stopwords) == 428

--- 

We can now write some basic text processing functions. A first step is to tokenize the text. You may use any tokenizer available in the `nltk` library:

In [19]:
# TODO: Implement this! (5 points)
nltk.download('punkt')

def tokenize(text):
    """
        Tokenize the text. 
        Input: text - a string
        Output: a list of tokens
    """
    
    return nltk.tokenize.word_tokenize(text)
    

[nltk_data] Downloading package punkt to /Users/berend/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
#####
text = "the quick brown fox jumps over the lazy dog"
tokens = tokenize(text)
print(tokens)
#####

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


---

*Answer the following questions*: 
- Why is stemming necessary, in particular for IR?
    - TODO: Answer this!
    - Stemming removes unnecessary information captures in suffixed and affixes. It makes the IR system more flexible and the IR task simpler as different word forms can be recognized as the same query word (stem. 
- Is there any setting (domain, scenario, etc) in which stemming can hurt performance? Illustrate with an example
    - *TODO: Answer this!*
    - Stemmers may generate false positives by overstemming (words that are stemmed to the same root while they should not have been), an example: university, universal and universe are all stemmed to "univers" (source: https://en.wikipedia.org/wiki/Stemming#Error_metrics). 
    - A domain which could suffer from errors in stemming would be a collection of medical texts. In medical texts, words with a Greek and Latin origin frequently occur. These words are conjugated differently than according to traditional English grammar. Example: Alumnus -> alumnu, alumni -> alumni, alumna -> alumna, this is an example of understemming. The word should have been stemmed to the same root, but they are not, as they do not follow standard English grammar. 
    Two different words may have the same without having the same semantical meaning. An example would be: 

Write a function to stem tokens. Again, you can use the `nltk` library for this

In [21]:
# TODO: Implement this! (5 points)
def stem_token(token):
    """
        Stem the given token, using any stemmer available from the nltk library
        Input: a single token
        Output: the stem of the token
    """
    
    stemmer = nltk.stem.PorterStemmer()
    
    return stemmer.stem(token)
    

In [22]:
####
print([stem_token(t) for t in tokens])
tokens_ = [
    'caresses', 'flies', 'dies', 'mules', 'denied',
    'died', 'agreed', 'owned', 'humbled', 'sized',
    'meeting', 'stating', 'siezing', 'itemization',
    'sensational', 'traditional', 'reference', 'colonizer',
    'plotted']
print([stem_token(t) for t in tokens_])
####

['the', 'quick', 'brown', 'fox', 'jump', 'over', 'the', 'lazi', 'dog']
['caress', 'fli', 'die', 'mule', 'deni', 'die', 'agre', 'own', 'humbl', 'size', 'meet', 'state', 'siez', 'item', 'sensat', 'tradit', 'refer', 'colon', 'plot']


---

*Answer the following questions*: 
- Another processing step (not done here) is to use n-grams. Illustrate why you would want to use n-grams in IR with an example.  
    - *TODO: Answer this!*
    - Unigram models assume that words are independent from their neighbours. However, in natural language this is rarely the case. Using N-grams can enable us to capture dependencies in word sequences. An example we have seen in class is the query "Paris Hilton". If we take this query as two separate words, we may find a Hilton hotel in Paris. However, when we consider the bi-gram "Paris Hilton", it is clear that we are looking for the celebrity Paris Hilton. 
- Usage of n-grams exacerbates some problems ex. in bi-gram language models. What is this problem? Suggest one solution 
    - *TODO: Answer this!*
    - When considering bi-grams rather than unigrams, the probability of encountering a new (unseen) bi-gram is higher than when considering unigrams. A solution could be to use laplace-smoothing, or backoff and interpolation (if bi-gram is unseen, try using the unigrams). 

--- 

The following function puts it all together. Given a string, it tokenizes it, and processes it according to the flags that you set.

In [23]:
#### Putting it all together
def process_text(text, stem=False, remove_stopwords=False, lowercase_text=False):
    
    tokens = []
    for token in tokenize(text):
        if remove_stopwords and token.lower() in stopwords:
            continue
        if stem:
            token = stem_token(token)
        if lowercase_text:
            token = token.lower()
        tokens.append(token)

    return tokens
#### 

Let's create two sets of pre-processed documents

In [24]:
# In this configuration:
# Don't preprocess the text, except to tokenize 
config_1 = {
  "stem": False,
  "remove_stopwords" : False,
  "lowercase_text": True
} 


# In this configuration:
# Preprocess the text: stem and remove stopwords
config_2 = {
  "stem": True,
  "remove_stopwords" : True,
  "lowercase_text": True, 
} 


We can now process the documents and queries according to the configuration specified above

In [25]:
####
doc_repr_1 = []
doc_repr_2 = []
for (doc_id, document) in docs:
    doc_repr_1.append((doc_id, process_text(document, **config_1)))
    doc_repr_2.append((doc_id, process_text(document, **config_2)))

####


--- 

## Section 2: Indexing (10 points)


A retrieval function usually takes in a query document pair, and scores a query against a document.  Our document set is quite small - just a few thousand documents. However, consider a web-scale dataset with a few million documents. In such a scenario, it would become infeasible to score every query and document pair. In such a case, we can build an inverted index. From Wikipedia:

> ... , an inverted index (also referred to as a postings file or inverted file) is a database index storing a mapping from content, such as words or numbers, to its locations in a table, .... The purpose of an inverted index is to allow fast full-text searches, at a cost of increased processing when a document is added to the database. ...


Consider a simple inverted index, which maps from word to document. This can improve the performance of a retrieval system significantly. In this assignment, we consider a *simple* inverted index, which maps a word to a set of documents. In practice, however, more complex indices might be used.  


### Building an index

A retrieval function usually takes in a query document pair, and scores a query against a document.  Our document set is quite small - just a few thousand documents. However, consider a web-scale dataset with a few million documents. In such a scenario, it would become infeasible to score every query and document pair. In such a case, we can build an inverted index. From Wikipedia:

> ... , an inverted index (also referred to as a postings file or inverted file) is a database index storing a mapping from content, such as words or numbers, to its locations in a table, .... The purpose of an inverted index is to allow fast full-text searches, at a cost of increased processing when a document is added to the database. ...


Consider a simple inverted index, which maps from word to document. This can improve the performance of a retrieval system significantly. In this assignment, we consider a *simple* inverted index, which maps a word to a set of documents. In practice, however, more complex indices might be used.  


---

In this assignment we will be using an index created in memory, since our dataset is tiny. To get started, build a simple index that maps simply counts the number of tokens present in a document. This index  is built using a python dictionary.

*Answer the following questions*:
- What is the time complexity of retrieving a list of documents from a python `dict` object? 
    - *TODO: Answer this!* 
    - Since Python's dictionaries use hash tables, a search of documents would have an average time complexity $O(1)$. 
- Consider the case with a 10 million documents. What is the time complexity of retrieval with an inverted index (assuming you can fit the entire index in memory)? (Hint: Consider length of a query $|q|$) 
    - *TODO: Answer this!*
    - First: q lookups in inverted index: $O(|q|)$. This yields k document IDs. Looking up the documents has a complexity of $k O(1)$. $kO(1) + O(|q|) = O(|q|)$. 
- For a large enough collection, we cannot store an index in memory. How is this tackled in practice (briefly explain)? Comment on the time complexity. 
    - *TODO: Answer this!*
    - The inverted list is split in several blocks that do fit in memory. Say we need B blocks to fit the entire list in memory, we would have to look up the query B times more.
- Mention a use-case in which a simple index (from word -> doc_id) might not suffice anymore. How would you modify the index to suit this use-case (if you can!)  
    - *TODO: Answer this!*
    - At some point, we migth want to index very large documents (e.g. books). Then, the fact that a term occurs in the document does not make it relevant per se. We can modify the index to contain counts or relative frequencies of the terms.
    
    
Now, implement a function to build an index:

In [29]:
# TODO: Implement this! 10 points
def build_tf_index(documents):
    """
    Build an inverted index (with counts). The output is a dictionary which takes in a token
    and returns a list of (doc_id, count) where 'count' is the count of the 'token' in 'doc_id'
    Input: a list of documents - (doc_id, tokens) 
    Output: An inverted index. [token] -> [(doc_id, token_count)]
    """
    
    index = defaultdict(list)
    
    for doc_id, doc in documents:
        tokens, counts = np.unique(doc, return_counts=True)
    
        for token, count in zip(tokens, counts):
            index[token].append((doc_id, count))
        
    
    return index
 
# Create the 2 indices
tf_index_1 = build_tf_index(doc_repr_1)
tf_index_2 = build_tf_index(doc_repr_2)

# This function returns the correct index 
def get_index(index_set):
    assert index_set in {1, 2}
    return {
        1: tf_index_1,
        2: tf_index_2
    }[index_set]

# This function correctly pre-processes the text given the index set
def preprocess_query(text, index_set):
    assert index_set in {1, 2}
    if index_set == 1:
        return process_text(text, **config_1)
    elif index_set == 2:
        return process_text(text, **config_2)
    

--- 

## Section 3: Retrieval  (80 points)

Now that we have cleaned and processed our dataset, we can start building simple IR systems. 

For now, we consider *simple* IR systems, which involve computing scores from the tokens present in the document/query. More advanced methods are covered in later assignments.

We will implement the following methods in this section:
- TF-IDF
- BM25
- Query Likelihood Models

--- 

### Ranking functions


Probably the simplest IR model is the Bag of Words (BOW) model. Implement a function that scores a query against a document using this model.   

Note that you can use either the count of the token or 'binarize' it i.e set the value equal to 1 if the token appears.   


**Note:** Make sure you use the `get_index` function to retrieve the correct index, and call `preprocess_query` with the correct index!

In [30]:
# TODO: Implement this! 10 points
def bow_search(query, index_set):
    """
        Perform a search over all documents with the given query. 
        Note #1: You have to use the `get_index` function created in the previous cells
        Note #2: You can binarize the counts if you wish to
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)    
    query = preprocess_query(query, index_set)
    scores = {}
    d_t = defaultdict(float)
    
    query_terms, query_counts = np.unique(query, return_counts=True)
    
    # compute q denominator
    q = (query_counts**2).sum()
    
    for doc_id, _ in docs:
        
        # compute numerator
        qd = 0
        for term, count in zip(query_terms, query_counts):
            doc_count = sum([j for i,j in index[term] if i == doc_id])
            qd += doc_count * count
            
        # compute cos-similarity (partly)
        scores[doc_id] = qd / np.sqrt(q)
    
    # compute d term
    for term in index:
        for doc_id, count in index[term]:
            d_t[doc_id] += count**2
                
    # update scores
    for doc_id in scores:
        scores[doc_id] /= np.sqrt(d_t[doc_id])
    
    # back to list and sort
    scores = [(k,v) for k,v in scores.items()]
    scores = sorted(scores, key = lambda x: x[1], reverse=True)
    
    return scores
    


*Answer the following questions*: 
- The BOW model is might not be a good choice for use in IR. Why? 
    - BOW assumes independence between words, and also neglects word order. However, word order might have a significant effect on the meaning and intent of the query, and we might lose information if we disregard the word order.

In [31]:
%%time
####
docs_by_id = dict(docs)
def print_results(docs, len_limit=50):    
    for i, (doc_id, score) in enumerate(docs):
        doc_content = docs_by_id[doc_id].strip().replace("\n", "\\n")[:len_limit] + "..."
        print(f"Rank {i}({score:.2}): {doc_content}")

test_bow = bow_search("report", index_set=1)[:5]
print(f"BOW Results:")
print_results(test_bow)
#### 

BOW Results:
Rank 0(0.45): ALGOL Sub-Committee Report - Extensions...
Rank 1(0.45): A Report Writer For COBOL...
Rank 2(0.45): A CRT Report Generating System...
Rank 3(0.41): Supplement to the ALGOL 60 Report...
Rank 4(0.38): Report on the Algorithmic Language ALGOL 60...
CPU times: user 119 ms, sys: 1.91 ms, total: 120 ms
Wall time: 120 ms


Before we implement the tf-idf scoring functions, let's first write a function to compute the document frequencies of all words.  

In [34]:
# TODO: Implement this! (5 points)
def compute_df(documents):
    """
        Compute the document frequency of all terms in the vocabulary
        Input: A list of documents
        Output: A dictionary with {token: document frequency)
    """
    
    doc_freq = defaultdict(int)
    
    for document in documents:
        for token in set(document):
            doc_freq[token] += 1

    return doc_freq
            
            
        
# get the document frequencies of each document
df_1 = compute_df([d[1] for d in doc_repr_1])
df_2 = compute_df([d[1] for d in doc_repr_2])

def get_df(index_set):
    assert index_set in {1, 2}
    return {
        1: df_1,
        2: df_2
    }[index_set]

Next, implement a function that computes a tf-idf score given a query.      

In [35]:
# TODO: Implement this! 10 points
def tfidf_search(query, index_set):
    """
        Perform a search over all documents with the given query using tf-idf. 
        Note #1: You have to use the `get_index` (and the `get_df`) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    df = get_df(index_set)
    query = preprocess_query(query, index_set)
    N = len(docs)
    
    scores = {}
    d_t = defaultdict(float)
    
    
    
    query_terms, query_counts = np.unique(query, return_counts=True)
    
    # compute q denominator
    q = sum([(query_counts[i] * np.log(N/df[term]))**2 for i,term in enumerate(query_terms)])
    
    for doc_id, _ in docs:
        
        # compute numerator
        qd = 0
        for term, count in zip(query_terms, query_counts):
            doc_count = sum([j for i,j in index[term] if i == doc_id])
            qd += doc_count * count * np.log(N/df[term])**2 ## CHECK HIER
            
        # compute cos-similarity (partly)
        scores[doc_id] = qd / np.sqrt(q)
    
    # compute d term
    for term in index:
        for doc_id, count in index[term]:
            d_t[doc_id] += (count*np.log(N/df[term]))**2
                
    # update scores
    for doc_id in scores:
        scores[doc_id] /= np.sqrt(d_t[doc_id])
    
    # back to list and sort
    scores = [(k,v) for k,v in scores.items()]
    scores = sorted(scores, key = lambda x: x[1], reverse=True)
    
    return scores
    
    
    
    
    

In [26]:
####
test_tfidf = tfidf_search("report", index_set=1)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf)
####

TFIDF Results:
Rank 0(0.46): A CRT Report Generating System...
Rank 1(0.44): Report on the Algorithmic Language ALGOL 60...
Rank 2(0.43): Report on the Algorithmic Language FORTRAN II...
Rank 3(0.43): A Report Writer For COBOL...
Rank 4(0.4): Supplement to the ALGOL 60 Report...


*Answer the following questions*: 
- It is generally not advisable to use the naive version of tf-idf. Why?
    - *TODO: Answer this!*
    - The naive version of tf-idf does not take into account the document length, which might become problematic if some of the documents become very long (e.g. books), since the query terms are more likely to appear more often in those long documents. Furthermore, if the query term does not occur in any of the documents, we encounter divisions by 0.  
- Illustrate with an example why using a sublinear scaling for TF is preferable in some cases.  
    - *TODO: Answer this!*
    - Intuitively, we can explain this by the fact that the 100th occurence of a term in a document is not as significant as the first or second time. In other words, by using a sublinear scaling we decrease the marginal significance of term occurences when the frequency increases.

--- 

*Answer the following questions*: 
- An alternative way to compute a query<>document score is to vectorize both the query and document (where each dimension corresponds to a token), and compute a score. The score can be computed using a dot product between the query and the document vectors. Why is the cosine similary function a better choice, particularly in IR? 
    - *TODO: Answer this!*
    - The cosine similarity is preferred over the dot product, as the cosine similarity is normalized. In other words, the cosine similarity only cares about the angle of difference between the vectors, whereas the dot product also cares about the magnitude. 
- What is the time complexity of a search if we are using the vector space method mentioned in the previous question? What is the time complexity if we're using an index (assume that it fits in memory)? Assume $N$ is the number of documents and $|q|$ is the length of a query. 
    - *TODO: Answer this!*

--- 

#### Query Likelihood Models

In this section you will implement a simple query likelihood model. 

First, let use implement a naive version of a QL model, assuming a multinomial unigram language model (with a uniform prior over the documents). 

**Note:** Make sure you use the `get_index` function to retrieve the correct index, and call `preprocess_query` with the correct index!

--- 

*Answer the following questions*: 
- Write down the formula for computing the query likelihood, assuming a multinomial unigram language model. 
    - *TODO: Answer this!*
- What problem does this naive method have? Suggest a simple way to fix it.
    - *TODO: Answer this!*

In [27]:
####
def doc_lengths(documents):
    doc_lengths = {doc_id:len(doc) for (doc_id, doc) in documents}
    return doc_lengths

doc_lengths_1 = doc_lengths(doc_repr_1)
doc_lengths_2 = doc_lengths(doc_repr_2)

def get_doc_lengths(index_set):
    assert index_set in {1, 2}
    return {
        1: doc_lengths_1,
        2: doc_lengths_2
    }[index_set]
####

In [28]:

# TODO: Implement this! 15 points
def naive_ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a naive QL model. 
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    doc_len = get_doc_lengths(index_set)
    query = preprocess_query(query, index_set)
    scores = {}
    
    for doc_id, _ in docs:
        doc_score = 1
        for term in query:
            freq = sum([c for i,c in index[term] if i==doc_id])
            doc_score *= freq/doc_len[doc_id]
        scores[doc_id] = doc_score
        
    # back to list and sort
    scores = [(k,v) for k,v in scores.items()]
    scores = sorted(scores, key = lambda x: x[1], reverse=True)
            
    return scores

In [29]:
####
test_naiveql = naive_ql_search("report", index_set=1)[:5]
print(f"TFIDF Results:")
print_results(test_naiveql)
####

TFIDF Results:
Rank 0(0.2): ALGOL Sub-Committee Report - Extensions...
Rank 1(0.2): A Report Writer For COBOL...
Rank 2(0.2): A CRT Report Generating System...
Rank 3(0.17): Supplement to the ALGOL 60 Report...
Rank 4(0.14): Report on the Algorithmic Language ALGOL 60...


Now, let's implement a (slightly more) complex QL model. This model should 'fix' the issue with the previous method. If your model requires hyperparameters, set a reasonable value. 

In [30]:
# TODO: Implement this! 20 points
def ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a appropriate QL model. 
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    doc_len = get_doc_lengths(index_set)
    query = preprocess_query(query, index_set)
    scores = {}
    C = sum(doc_len.values())
    
    # rank according to Jelinek-Mercer method (with parameter alpha)
    for doc_id, _ in docs:
        doc_score = 0
        for term in query:
            freq_doc = sum([c for i,c in index[term] if i==doc_id])
            freq_col = sum([c for i,c in index[term]])
            doc_score += np.log((1-alpha) * freq_doc/doc_len[doc_id] * alpha * freq_col/C)
        scores[doc_id] = doc_score
        
    # back to list and sort
    scores = [(k,v) for k,v in scores.items()]
    scores = sorted(scores, key = lambda x: x[1], reverse=True)
            
    return scores

In [31]:
#### Test the QL model
alpha = 0.7
test_ql_results = ql_search("report", index_set=1)[:5]
print_results(test_ql_results)
print()
test_ql_results_long = ql_search("report " * 10, index_set=1)[:5]
print_results(test_ql_results_long)
####

/Users/TomLotze/miniconda3/envs/IR1-hw1/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log


Rank 0(-1.1e+01): ALGOL Sub-Committee Report - Extensions...
Rank 1(-1.1e+01): A Report Writer For COBOL...
Rank 2(-1.1e+01): A CRT Report Generating System...
Rank 3(-1.1e+01): Supplement to the ALGOL 60 Report...
Rank 4(-1.1e+01): Report on the Algorithmic Language ALGOL 60...

Rank 0(-1.1e+02): ALGOL Sub-Committee Report - Extensions...
Rank 1(-1.1e+02): A Report Writer For COBOL...
Rank 2(-1.1e+02): A CRT Report Generating System...
Rank 3(-1.1e+02): Supplement to the ALGOL 60 Report...
Rank 4(-1.1e+02): Report on the Algorithmic Language ALGOL 60...


*Answer the following questions*: 
- What happens to the query likelihood for long queries? What is a simple fix for this issue?
    - *TODO: Answer this!*


--- 

#### BM25

In this section, we will implement the widely used and hard to beat BM25 scoring function. 


In [69]:
# TODO: Implement this! (20 points)
k1 = 1.2
b = 0.75
N = len(docs)


def bm25_search(query, index_set):
    """
        Perform a search over all documents with the given query using BM25. 
        Note #1: You have to use the `get_index` (and `get_doc_lengths`) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
        
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    
    query = preprocess_query(query, index_set)
    dl_avg = np.mean(list(doc_lengths.values()))
    scores = {}
    
    for doc_id, _ in docs:
        dl = doc_lengths[doc_id]
        doc_score = 0
        for term in query:
            tf =  sum([c for i,c in index[term]]) # term frequency
            df = sum([c for i,c in index[term] if i==doc_id]) # document frequency of term t
            #print(f'tf: {tf}, df: {df}')
            if df == 0:
                term_score = 0
            else:
                term_score = np.log(N/df) * (((k1 + 1) * tf) / (k1 * ((1-b) + b * (dl/dl_avg)) + tf))
            doc_score += term_score
        scores[doc_id] = doc_score
        
    
    # back to list and sort
    scores = [(k,v) for k,v in scores.items()]
    ranking = sorted(scores, key = lambda x: x[1], reverse=True)
    
    
    
    return ranking # this is a list of tuples, ranked on relevance (doc_id, score)

In [79]:
#### Test the BM25 model
test_bm25_results = bm25_search("science", index_set=1)[:5]
print_results(test_bm25_results)
####

Rank 0(1.8e+01): The Practical Aspect of Computer Science Education...
Rank 1(1.8e+01): The GRE Advanced Test in Computer Science...
Rank 2(1.8e+01): Production and Employment of Ph.D.'s in Computer S...
Rank 3(1.8e+01): Computer Science as Empirical Inquiry: Symbols and...
Rank 4(1.8e+01): A Comment on the Practical Aspects of Computer Sci...




*Answer the following questions*: 
- Briefly explain how the BM25 model improves upon the tf-idf model.
    - *TODO: Answer this!*
    
---

In [80]:
#### Highlighter function
# class for results
ResultRow = namedtuple("ResultRow", ["doc_id", "snippet", "score"])
# doc_id -> doc
docs_by_id = dict((d[0], d[1]) for d in docs)

def highlight_text(document, query, tol=17):
    import re
    tokens = tokenize(query)
    regex = "|".join(f"(\\b{t}\\b)" for t in tokens)
    regex = re.compile(regex, flags=re.IGNORECASE)
    output = ""
    i = 0
    for m in regex.finditer(document):
        start_idx = max(0, m.start() - tol)
        end_idx = min(len(document), m.end() + tol)
        output += "".join(["...",
                        document[start_idx:m.start()],
                        "<strong>",
                        document[m.start():m.end()],
                        "</strong>",
                        document[m.end():end_idx],
                        "..."])
    return output.replace("\n", " ")


def make_results(query, search_fn, index_set):
    results = []
    for doc_id, score in search_fn(query, index_set):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results
####

---
---

The widget below allows you to play with the search functions you've written so far. This can be used, for example, to answer some of the theory questions

In [1]:
# Set this to the function you want to test
# this function should take in a query (string)
# and return a sorted list of (doc_id, score) 
# with the most relevant document in the first position
search_fn = bm25_search
index_set = 1

text = widgets.Text(description="Search Bar", width=200)
display(text)

def handle_submit(sender):
    print(f"Searching for: '{sender.value}'")
    
    results = make_results(sender.value, search_fn, index_set)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit)

NameError: name 'bm25_search' is not defined

## Section 4: Offline Evaluation (45 points)

Before we jump in and implement an algorithm for retrieval, we first have to learn how to evaluate such a system. In particular, we will work with offline evaluation metrics. These metrics are computed on a dataset with known relevance judgements.

Implement the following evaluation metrics. 

1. Precision
2. Recall
3. Mean Average Precision
4. Expected Reciprocal Rank

---
*Answer the following questions*: 
- What are the main limitations of an offline evaluation?
    - *TODO: Answer this!*

---

Let's take a look at the `qrels.text` file, which contains the ground truth relevance scores. The relevance labels for CACM are binary - either 0 or 1. 


In [82]:
!head ./datasets/qrels.text

01 1410  0 0
01 1572  0 0
01 1605  0 0
01 2020  0 0
01 2358  0 0
02 2434  0 0
02 2863  0 0
02 3078  0 0
03 1134  0 0
03 1613  0 0


The first column is the `query_id` and the second column is the `document_id`. You can safely ignore the 3rd and 4th columns. Write a function to read in the file: 

In [83]:
# TODO: Implement this!
def read_qrels(root_folder = "./datasets/"):
    """
        Reads the qrels.text file. 
        Output: A dictionary: query_id -> [list of relevant documents]
    """
    output = {}
    filepath = os.path.join(root_folder, 'qrels.text')
    
    with open(filepath, 'r') as f:
        lines = f.read().splitlines()

    for line in lines:
        query_id, doc_id, _, _ = line.split()

        query_id = int(query_id)
        doc_id = int(doc_id)

        if query_id not in output:    
            output[query_id] = [doc_id]
        else:
            output[query_id].append(doc_id)
            
    return output
    

qrels = read_qrels()

In [84]:
####
assert len(qrels) == 52, "There should be 52 queries with relevance judgements"
assert sum(len(j) for j in qrels.values()) == 796, "There should be a total of 796 Relevance Judgements"
####

Now, implement the metrics below. 

**Note:** For a given query `query_id`, you can assume that documents *not* in `qrels[query_id]` are not relevant to `query_id`. 


In [87]:
# TODO: Implement this! (10 points)
def recall_k(results, relevant_docs, k):
    """
        Compute Recall@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Recall@K
    """
    pred_doc_ids = [doc_id for doc_id, _ in results[:k]]
    correct = len(set(relevant_docs) & set(predicted_doc_ids))
    
    return correct/len(relevant_docs)
    
    
    
# TODO: Implement this! (10 points)
def precision_k(results, relevant_docs, k):
    """
        Compute Precision@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), 
                    with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Precision@K
    """
    
    pred_doc_ids = [doc_id for doc_id, _ in results[:k]]
    correct = len(set(relevant_docs) & set(predicted_doc_ids))
    
    return correct / k
    
    

# TODO: Implement this! (10 points)
def average_precision(results, relevant_docs):
    """
        Compute Average Precision (for a single query - the results are 
        averaged across queries to get MAP in the next few cells)
        Hint: You can use the recall_k and precision_k functions here!
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: Average Precision
    """
    
    ap = 0
    
    for i, (doc_id, _) in enumerate(results):
        if doc_id in relevant_docs:
            ap += precision_k(results, relevant_docs, i+1) / (i+1)
            


# TODO: Implement this! (15 points)
def err(results, relevant_docs):
    """
        Compute the expected reciprocal rank.
        Hint: https://dl.acm.org/doi/pdf/10.1145/1645953.1646033?download=true
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: ERR
        
    """
    g = np.array([score for doc_id, score in results])
    R_g = (np.power(2, g)-1) / (np.max(g))
    n = len(g)
    err = np.zeros(n)
    
    for r in range(1, n+1):
        err[r-1] = 1/r * np.prod(R_g[:r-1]) * R_g[r-1]
        
    return err.sum()
####

*Answer the following questions*: 
- What are the main drawbacks of precision & recall?
    - *TODO: Answer this!*
- What problems with Precision@K does Average Precision solve? 
    - *TODO: Answer this!*
- The CACM dataset has *binary* relevance judgements. However, a more suitable way of assigning judgements is to use graded relevance. Mention a metric which might be more suitable for a graded relevance, and breifly explain why. 
    - *TODO: Answer this!*
- Consider a text processing step: stemming. What effect does this have on metrics? (Hint: Try changing the pre-processing config and try it out!)
    - *TODO: Answer this!*

---

Let's define some metrics@k using [partial functions](https://docs.python.org/3/library/functools.html#functools.partial)

In [88]:
####
recall_at_1 = partial(recall_k, k=1)
recall_at_5 = partial(recall_k, k=5)
recall_at_10 = partial(recall_k, k=10)
precision_at_1 = partial(precision_k, k=1)
precision_at_5 = partial(precision_k, k=5)
precision_at_10 = partial(precision_k, k=10)
####

---

The following function evaluates a `search_fn` using the `metric_fn`. Note that the final number is averaged over all the queries

In [89]:
####
def evaluate_search_fn(search_fn, metric_fn, index_set):
    # build a dict query_id -> query 
    queries_by_id = dict((q[0], q[1]) for q in queries)
    
    metrics = np.zeros(len(qrels), dtype=np.float32)
    for i, (query_id, relevant_docs) in enumerate(qrels.items()):
        query = queries_by_id[query_id]
        results = search_fn(query, index_set)
        metrics[i] = metric_fn(results, relevant_docs)
    
    return metrics.mean()
####

In [ ]:
index_sets = {1, 2}

list_of_metrics = [
    ("ERR", err),
    ("MAP", average_precision),
    ("Recall@1",recall_at_1),
    ("Recall@5", recall_at_5),
    ("Recall@10", recall_at_10),
    ("Precision@1", precision_at_1),
    ("Precision@5", precision_at_5),
    ("Precision@10", precision_at_10)]

list_of_search_fns = [
    ("NaiveQL", naive_ql_search),
    ("QL", ql_search),
    ("BM25", bm25_search),
    ("BOW", bow_search),
    ("TF-IDF", tfidf_search)
]


results = {}
for index_set in index_sets:
    results[index_set] = {}
    print(f"Index: {index_set}")
    for search_fn_name, search_fn in list_of_search_fns:
        print(f"\tEvaluating Search Function: {search_fn_name}")
        results[index_set][search_fn_name] = {}
        for metric_name, metric_fn in list_of_metrics:
            r = evaluate_search_fn(search_fn, metric_fn, index_set).mean()
            print(f"\t\tMetric: {metric_name}: {r}")
            results[index_set][search_fn_name][metric_name] = r
        print()

## Section 5: Results and Analysis (20 points)

The `results` dictionary contains the results for all search functions we implemented. Plot the results in bar charts, with clear labels:

In [ ]:
# TODO: Implement this! (20 points)

Write a summary of what you observe in the results.
You summary should compare results across the 2 indices and the methods being used. State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations.      
*Hint*: You may build upon the answers from the previous sections. 

*TODO: Answer this!*